# huntabyte/shadcn-svelte

> 

In [1]:
#| default_exp huntabyte-shadcn-svelte

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export

from lovely_docs.settings import settings, GitSource

from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
from pathlib import Path
import logging
import subprocess

from lovely_docs.git import clone_repo

In [4]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
#| export

build = True

source = GitSource(
    name="huntabyte/shadcn-svelte",
    doc_dir="docs/.svelte-kit/cloudflare/docs",
    repo="https://github.com/huntabyte/shadcn-svelte",
    commit="main",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

In [6]:
commit, clone_dir

('2c94bbbe096fcebb49282369b8a91a7e2e73eb75',
 Path('../git_dir/github.com/huntabyte/shadcn-svelte'))

In [7]:
if build:
    with open("huntabyte-shadcn-svelte.log", "w") as log_file:
        # Clean the repo
        subprocess.run(["git", "clean", "-dfx"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in root
        subprocess.run(["pnpm", "install"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Build the package and docs
        subprocess.run(["pnpm", "build"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # # Install dependencies in docs
        # subprocess.run(["pnpm", "install"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

        # # Build the docs
        # subprocess.run(["pnpm", "build"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

In [8]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)

In [9]:
Path("shadcn-svelte.json").write_text(tree.model_dump_json(indent=2))

586007

In [ ]:
# tree.subdirs = tree.subdirs[:3]
# tree.subdirs[1].subdirs = tree.subdirs[1].subdirs[:4]

#### Split the single llms.txt file into pages and directories based on heading levels

In [ ]:
# compacted_tree = compact_tree(tree)

In [ ]:
# Path("shadcn-svelte-compact.json").write_text(compacted_tree.model_dump_json(indent=2))

In [10]:
extra = "In cli examples, when multiple tools present (e.g npm, pnpm, bun), pick just one - the most common one"

processed_tree = await process_tree_depth_first(settings, tree, source.name, extra, extra)

+208.039s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/accordion.md
+0.261s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/alert-dialog.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/alert.md
+0.018s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/aspect-ratio.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/avatar.md
+0.017s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/badge.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/breadcrumb.md
+0.016s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/button-group.md
+0.018s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/button.md
+0.021s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/calendar.md
+0.022s DEBUG lovely_docs/docs.py:128 llm_process_page Processing components/card.md
+0.024s DEBUG love

In [ ]:
Path("shadcn-svelte-processed.json").write_text(processed_tree.model_dump_json(indent=2))

#### Calculate cost

In [11]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 318,584
  Output tokens: 76,144
  Total tokens: 394,728
Cost:
  Total: $0.70
  Input: $0.32
  Output: $0.38


#### save the results

In [12]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)

In [ ]:
processed_tree.children